<a href="https://colab.research.google.com/github/mariasuescum/G3-COMPUTER-VISION/blob/feat%2Fnotebook-training/training_dataset_logo3_v2_2_0_yolov8s_obb_b8_w4_img640_ep80_colab_run3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Entrenamiento de YOLOv8s en Google Colab

Este notebook está configurado con parámetros conservadores para asegurar la finalización del entrenamiento en un entorno con recursos limitados y un plazo ajustado.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 30.0 MB/s eta 0:00:00


In [ ]:
import os
import tarfile
from ultralytics import YOLO

# --- PARÁMETROS DE CONFIGURACIÓN DE EMERGENCIA ---
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
DRIVE_PROJECT_PATH = '/content/drive/MyDrive/Colab Notebooks/mod03-projects/project-xii-computer-vision'

# --- Dataset (Comprimido) ---
COMPRESSED_DATASET_NAME = 'dataset_v2_yolov8_obb.tar.gz'
COMPRESSED_DATASET_PATH = os.path.join(DRIVE_PROJECT_PATH, COMPRESSED_DATASET_NAME)
DATASET_BASE_PATH = '/content/dataset_v2_yolov8_obb'
DATASET_YAML_PATH = os.path.join(DATASET_BASE_PATH, 'data.yaml')

# --- Lógica de Descompresión ---
if not os.path.exists(DATASET_BASE_PATH):
    print(f"Dataset no encontrado en {DATASET_BASE_PATH}. Descomprimiendo...")
    if not os.path.exists(COMPRESSED_DATASET_PATH):
        raise FileNotFoundError(f"El archivo {COMPRESSED_DATASET_PATH} no existe.")
    with tarfile.open(COMPRESSED_DATASET_PATH, 'r:gz') as tar_ref:
        tar_ref.extractall('/content/')
    print("Descompresión completada.")
else:
    print(f"Dataset ya encontrado en {DATASET_BASE_PATH}.")

# --- Configuración del Modelo y Entrenamiento ---
MODEL_NAME = 'yolov8s.pt'
EPOCHS = 80
RUN_NAME = 'yolov8s_obb_b8_w4_img640_ep80_colab_run3'
BATCH_SIZE = 8
IMG_SIZE = 640
WORKERS = 4
PATIENCE = 20
CACHE = False
OPTIMIZER = 'AdamW'
COS_LR = True

print('--- CONFIGURACIÓN DE ENTRENAMIENTO ---')
print(f'Modelo: {MODEL_NAME}')
print(f'Epochs: {EPOCHS}')
print(f'Batch Size: {BATCH_SIZE}')
print(f'Image Size: {IMG_SIZE}')
print('----------------------------------------')
print('--- COMPROBACIONES DE DATASET, YAML Y MODELO ---')
print(f"Dataset listo para usar en: {DATASET_BASE_PATH}")
print(f"Archivo YAML: {DATASET_YAML_PATH}")
print(f"Modelo base: {MODEL_NAME}")
print('----------------------------------------')

# Cargar el modelo
model = YOLO(MODEL_NAME)

# Entrenar
results = model.train(
    data=DATASET_YAML_PATH,
    project=os.path.join(DRIVE_PROJECT_PATH, 'training_results'),
    name=RUN_NAME,
    epochs=EPOCHS,
    patience=PATIENCE,
    batch=BATCH_SIZE,
    imgsz=IMG_SIZE,
    workers=WORKERS,
    cache=CACHE,
    optimizer=OPTIMIZER,
    cos_lr=COS_LR,
    device='0',
    plots=True,
    save_json=True
)

print('--- ENTRENAMIENTO FINALIZADO ---')
print(f'Resultados guardados en: {results.save_dir}')

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Dataset no encontrado en /content/dataset_v2_yolov8_obb. Descomprimiendo...


/tmp/ipython-input-2788950871.py:21: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar_ref.extractall('/content/')


Descompresión completada.
--- CONFIGURACIÓN DE ENTRENAMIENTO ---
Modelo: yolov8s.pt
Epochs: 80
Batch Size: 8
Image Size: 640
----------------------------------------
--- COMPROBACIONES DE DATASET, YAML Y MODELO ---
Dataset listo para usar en: /content/dataset_v2_yolov8_obb
Archivo YAML: /content/dataset_v2_yolov8_obb/data.yaml
Modelo base: yolov8s.pt
----------------------------------------
Ultralytics 8.3.193 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/dataset_v2_yolov8_obb/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=80, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False,

In [ ]:
# Cargar el mejor modelo guardado durante el entrenamiento
best_model_path = os.path.join(DRIVE_PROJECT_PATH, 'training_results', RUN_NAME, 'weights', 'best.pt')
model = YOLO(MODEL_NAME)

# Evaluar en el conjunto de validación
metrics = model.val()

print('--- MÉTRICAS DE VALIDACIÓN ---')
print(f'mAP50-95: {metrics.box.map:.4f}')
print(f'   mAP50: {metrics.box.map50:.4f}')
print(f'   mAP75: {metrics.box.map75:.4f}')